In [1]:
# imports
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


## MongoDB

### Set up remote database

In [3]:
# read
with open("../.nosync/mongoDB.json", "r") as file:
    credentials = json.load(file)

uri = (
    "mongodb+srv://medvetslos:"
    + credentials["pwd"]
    + "@ind320-project.lunku.mongodb.net/?retryWrites=true&w=majority&appName=IND320-project"
)

mdb_client = MongoClient(uri, server_api=ServerApi("1"))

try:
    mdb_client.admin.command("ping")
    print("Pinged your deployment. Successfully connected to MongoDB.")
except Exception as exceptionMsg:
    print(exceptionMsg)

Pinged your deployment. Successfully connected to MongoDB.


In [3]:
# Creating collections for municipality data and gas prices
## Todo:
# - Lage en sjekk for eksisterende collections 

database = mdb_client["IND320-project"]
collection_names = ["muncipalities", "gas"]

# def collection_check(database, collection_name):
    # try:
    #     database.validate_collection(collection_name)
    # except pymongo.errors.OperatiosFailure:
    #     print(f"{collection_name} does not exist as a collection.")

municipalities = database["municipalities"]
gas = database["gas"]


## Cassandra

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()
keyspace = "new_table"
session.execute("CREATE KEYSPACE IF NOT EXISTS" + " " + keyspace + " " + 
                "WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1};")


## Webscraping



In [46]:
webscrape_url = "https://en.wikipedia.org/wiki/List_of_municipalities_of_Denmark"

## Todo: 
# - write a check to not insert if records already exists.

page = requests.get(webscrape_url)
soup = BeautifulSoup(page.content, "html.parser")
wiki_table = soup.find("table", attrs={"class": "wikitable sortable"})

df_municipalities = pd.read_html(StringIO(str(wiki_table)))[0]
records_municipalities = df_municipalities.to_dict("records")




# Writing from pandas to MongoDB
database["municipalities"].insert_many(records_municipalities)

InsertManyResult([ObjectId('66f444ee78ba03e2518e376d'), ObjectId('66f444ee78ba03e2518e376e'), ObjectId('66f444ee78ba03e2518e376f'), ObjectId('66f444ee78ba03e2518e3770'), ObjectId('66f444ee78ba03e2518e3771'), ObjectId('66f444ee78ba03e2518e3772'), ObjectId('66f444ee78ba03e2518e3773'), ObjectId('66f444ee78ba03e2518e3774'), ObjectId('66f444ee78ba03e2518e3775'), ObjectId('66f444ee78ba03e2518e3776'), ObjectId('66f444ee78ba03e2518e3777'), ObjectId('66f444ee78ba03e2518e3778'), ObjectId('66f444ee78ba03e2518e3779'), ObjectId('66f444ee78ba03e2518e377a'), ObjectId('66f444ee78ba03e2518e377b'), ObjectId('66f444ee78ba03e2518e377c'), ObjectId('66f444ee78ba03e2518e377d'), ObjectId('66f444ee78ba03e2518e377e'), ObjectId('66f444ee78ba03e2518e377f'), ObjectId('66f444ee78ba03e2518e3780'), ObjectId('66f444ee78ba03e2518e3781'), ObjectId('66f444ee78ba03e2518e3782'), ObjectId('66f444ee78ba03e2518e3783'), ObjectId('66f444ee78ba03e2518e3784'), ObjectId('66f444ee78ba03e2518e3785'), ObjectId('66f444ee78ba03e2518e37

In [47]:
database["municipalities"].find_one()

{'_id': ObjectId('66f444ee78ba03e2518e376d'),
 'LAU-1 code 1': 101,
 'Municipality': 'Copenhagen',
 'Administrative Center': 'Copenhagen',
 'Total Area (km²)': 86.6,
 'Population (2012-01-01)': 549050,
 'Region': 'Capital'}